In [40]:
import os
import pandas as pd
import nltk
import string
import contractions

In [41]:
stop_words = set(nltk.corpus.stopwords.words('english'))

In [42]:
albums = ["Taylor Swift", "Fearless (Taylor's Version)", "Speak Now (Taylor's Version)", 
          "Red (Taylor's Version)", "1989 (Taylor's Version)", "reputation", "Lover", 
          "Midnights", "folklore", "evermore", "The Tortured Poets Department"]

In [43]:
df = pd.read_csv('songs.csv')


In [44]:
in_album = df[df['Album'].isin(albums)]
df = in_album

In [45]:
lemmatizer = nltk.WordNetLemmatizer()

within_description = 0
for lyric in df['Lyrics']:
    word_list=lyric.split()
    word_data = ""
    for word in word_list:
        if "[" or "]" in word:
            if "[" in word and not("]" in word):
                within_description = 1
            elif "]" in word and not("[" in word):
                within_description = 0
            elif "[" in word and "]" in word:
                within_description = 0
            if "(" in word and not(")" in word):
                within_description = 1
            elif ")" in word and not("(" in word):
                within_description = 0
            elif "(" in word and ")" in word:
                within_description = 0
        if not("]" in word or "[" in word or "(" in word or ")" in word) and not(within_description):
            for contraction in contractions.contractions_dict.keys():
                if contraction.lower() in word.lower():
                    word = contractions.fix(word)
            word_data += word + " "

    nltk_tokens = nltk.word_tokenize(text=word_data, language='english', preserve_line=True)
    
    nltk_tokens = [token.lower() for token in nltk_tokens if token.lower() not in stop_words]
    nltk_tokens = [token for token in nltk_tokens if token not in string.punctuation]
    nltk_tokens = [lemmatizer.lemmatize(token) for token in nltk_tokens]
    nltk_tokens = [token for token in nltk_tokens if token != "''" and 
                   token != "``" and token != "'s" and token != "'d" and token != "oh"
                   and token != "hey" and token != "'m" and token != "mm" and token != "oh-oh"
                   and token != "oh-oh-oh" and token != "mmm" and token != "ooh" and token != "ooh-oh"
                   and token != "mm-mm" and token != "la" and token != "ah-ah" and token != "ha"
                   and token != "ha-ah" and token != "di" and token != "da" and token != "da-da" and token != "uh-huh"
                   and token != "ra-di-di-di-di-di-di-di-di-di-da-da"]

    lyric_column = df['Lyrics'] == lyric
    row_index = df.index[lyric_column]
    title = df[df['Lyrics'] == lyric]['Title'].values[0]
    album = df[df['Lyrics'] == lyric]['Album'].values[0]
    file_path = 'Songs/'+ album
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    song = os.path.join(file_path, f"{title}.txt")
    if "(Remix)" not in song:
        with open(song, "w", encoding="utf-8") as f:
            literal = "\n".join(nltk_tokens)
            f.write(literal)

